# Uncertainties Example

In [ ]:
from IPython.display import display, HTML
display(HTML('<a target="_blank" href="https://colab.research.google.com/github/WaterFutures/EPyT-Flow/blob/main/docs/examples/uncertainties.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>'))

This example demonstrates how apply uncertainties to model parameters such as demand patterns.

In [ ]:
%pip install epyt-flow --quiet

In [ ]:
import numpy as np
from epyt_flow.data.networks import load_ltown
from epyt_flow.simulation import ScenarioSimulator, ModelUncertainty, \
    RelativeUniformUncertainty
from epyt_flow.utils import to_seconds, plot_timeseries_data

Load the [L-Town network](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.data.html#epyt_flow.data.networks.load_ltown) with realistic demands:

In [ ]:
network_config = load_ltown(include_default_sensor_placement=True,
                            use_realistic_demands=True, verbose=False)

Create a new [simulation](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.html#epyt_flow.simulation.scenario_simulator.ScenarioSimulator):

In [ ]:
sim = ScenarioSimulator(scenario_config=network_config)

Set simulation duration to two hours:

In [ ]:
sim.set_general_parameters(simulation_duration=to_seconds(hours=2))

Add [uncertainty](https://epyt-flow.readthedocs.io/en/stable/tut.uncertainty.html) (i.e. randomness) with respect to the demand pattern -- i.e. demand pattern values can deviate up to 25% from their original value.
Consequently, the simulation is no longer deterministic and the results vary from run to run.

Here, we use [RelativeUniformUncertainty](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.uncertainty.html#epyt_flow.uncertainty.uncertainties.RelativeUniformUncertainty) for the demand pattern uncertainty and set it by calling [set_model_uncertainty()](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.html#epyt_flow.simulation.scenario_simulator.ScenarioSimulator.set_model_uncertainty):

In [ ]:
uc = RelativeUniformUncertainty(low=0.75, high=1.25)

sim.set_model_uncertainty(ModelUncertainty(demand_pattern_uncertainty=uc))

Run simulation three times and retrieve sensor readings at node "n105":

In [ ]:
measurements = []
for _ in range(3):
    scada_data = sim.run_simulation()
    measurements.append(scada_data.get_data_pressures(sensor_locations=["n105"]).
                        flatten().tolist())

plot_timeseries_data(np.array(measurements),
                     x_axis_label="Time (5min steps)", y_axis_label="Pressure in $m$")

Do not forget to close the simulation!

In [ ]:
sim.close()